In [1]:
import os, random
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from model import *
from utils import *
from data_split_subsys import dji_dataloader


import warnings
warnings.filterwarnings(action='ignore')

# Pytorch determistic
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In C:\Users\kjinb\anaconda3\envs\TS\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\kjinb\anaconda3\envs\TS\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\kjinb\anaconda3\envs\TS\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\kjinb\anaconda3\envs\TS\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\kjinb\anaconda3\envs\TS\lib\site-packages\matplo

In [2]:
def split_behavior(df, behavior):
    behavior_index_value = df[df['0_behavior'] == behavior].index.values
    idx_for_behavior = np.where(behavior_index_value[1:] - behavior_index_value[:-1] > 1)[0] + 1
    return idx_for_behavior

def get_scenario(data):   
    prepare_idx  = split_behavior(data, 'prepare')
    takeoff_idx  = split_behavior(data, 'takeoff')
    forward_idx  = split_behavior(data, 'forward')
    backward_idx = split_behavior(data, 'backward')
    right_idx    = split_behavior(data, 'right')
    left_idx    = split_behavior(data, 'left')
    up_idx       = split_behavior(data, 'up')
    down_idx     = split_behavior(data, 'down')
    land_idx     = split_behavior(data, 'land')
    behavior_idx = [prepare_idx, takeoff_idx, forward_idx, backward_idx, right_idx, left_idx, up_idx, down_idx, land_idx] 
    scenarios = [np.sum(np.asarray(behavior_idx)[:, each_idx]) for each_idx in range(len(prepare_idx))]
    scenarios.append(len(data))
    
    scenario_list = [None for _ in range(len(data))]
    start = 0
    for scenario_num, scenario_idx in enumerate(scenarios):
        for i in range(start, scenario_idx):
            scenario_list[i] = scenario_num
        start = scenario_idx        
    return scenario_list, scenarios

def get_subsystems(all_columns, target_colum):
    system_cate = [each.split('.')[0] for each in all_columns]
    selected_subsystem = all_columns[np.array(system_cate) == target_colum]
    return list(selected_subsystem) + ['timestamp']

def normalize_v1(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    print(random_scenario)
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]
    
    train_scaler   = []
    val_scaler   = []
    test_scaler   = []
    
    train_normalized = []
    val_normalized   = []
    test_normalized  = []
    
    for i in np.unique(data['scenario']):
        scaler          = CustomScaler()
        target_data     = data[data['scenario'] == i].drop(['scenario'], axis=1)
        not_norm_column = ison_list.copy()
        not_norm_column.append('0_behavior')
        target_column   = set(target_data.keys()).difference(set(not_norm_column))
        
        if i in train_scenario:
            train_normalized.append(scaler.fit_transform(target_data, target_column))
            train_scaler.append(scaler)
        elif i in val_scenario:
            val_normalized.append(scaler.fit_transform(target_data, target_column))
            val_scaler.append(scaler)
        elif i in test_scenario:
            test_normalized.append(scaler.fit_transform(target_data, target_column))
            test_scaler.append(scaler)
    
    train_norm = np.concatenate(train_normalized)
    val_norm = np.concatenate(val_normalized)
    test_norm = np.concatenate(test_normalized)

    return [train_norm, train_scenario, train_scaler],[val_norm, val_scenario, val_scaler],[test_norm, test_scenario, test_scaler]

def normalize_v2(data, ison_list, train=50, val=5, test=10):
    random_scenario = np.random.permutation(np.unique(data['scenario']))

    train_scenario = random_scenario[:train]
    val_scenario = random_scenario[train:train + val]
    test_scenario = random_scenario[train + val:train + val + test]

    random_scenario = np.random.permutation(np.unique(data['scenario']))

    train_scenario = random_scenario[:train]
    val_scenario = random_scenario[train:train+val]
    test_scenario = random_scenario[train+val:train+val+test]

    train_idx = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in train_scenario]
    train_data = data.iloc[sorted(np.hstack(train_idx))]

    val_idx = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in val_scenario]
    val_data = data.iloc[sorted(np.hstack(val_idx))]

    test_idx = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in test_scenario]
    test_data = data.iloc[sorted(np.hstack(test_idx))]

    not_norm_column = ison_list.copy()
    not_norm_column.append('0_behavior')
    not_norm_column.append('scenario')
    target_column = set(train_data.keys()).difference(set(not_norm_column))

    scaler = CustomScaler()
    scaler.fit_transform(train_data, target_column)
    scaler.transform(val_data, target_column)
    scaler.transform(test_data, target_column)

    return [train_data, val_data, test_data], \
           [train_scenario, val_scenario, test_scenario], \
           scaler

def get_data():
    data = pd.read_csv('dji_normal_processed.csv')
    data.drop('AAClock', axis=1, inplace=True)

    # Subsystem의 ison 변수 위치 재파악
    ison_list = [each for each in list(data.keys()) if each.find('ison') > 0]
    ison_dict = {key: np.where(np.array(list(data.keys())) == key)[0][0] for key in ison_list}

    (train_data, val_data, test_data), (train_scenario, val_scenario, test_scenario), scaler = normalize_v2(data, ison_list, train=20, val=3, test=10)


    # 시나리오별 성능 확인용
    train_scenario_index = train_data['scenario']
    val_scenario_index   = val_data['scenario']
    test_scenario_index  = test_data['scenario']

    X_train, y_train = split_series_v2(train_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_val, y_val = split_series_v2(val_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_test, y_test = split_series_v2(test_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)

    X_train_drop = X_train[:, :, :]
    X_val_drop = X_val[:, :, :]
    X_test_drop = X_test[:, :, :]
    
    res_list = list(train_data.drop(['scenario'], axis=1).columns)
    res_list_ison_out = [each for each in res_list if not each.find('ison')>0]
    res_list_ison_out_dict = {each:idx for idx, each in enumerate(res_list_ison_out)}

    meta = [each.split('.') for each in res_list_ison_out]
    meta_head = np.array([each.split('.')[0] for each in res_list_ison_out])
    super_column = np.unique(meta_head)

    column_idx = {}
    for each_column in super_column:
        column_idx[each_column] = list(np.where(meta_head == each_column)[0])
    
    return X_train_drop, y_train, X_val_drop, y_val, X_test_drop, y_test, column_idx

In [5]:
def train(model, schedule, EPOCH, train_dataloader, val_dataloader, SAVE_PATH, model_type, device):
    print('train path: ', SAVE_PATH+'/dji_res.csv')
    
    scaler = torch.cuda.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterionMSE = nn.MSELoss()
    if schedule:
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1)
        
    res = {
        'train_mse':[],
        'val_mse':[],
        'best_total':[],
    }
    
    best_total = np.inf
    best_epoch = 0
    
    for epoch in trange(EPOCH):
        # Training Procedure
        train_running_step = len(train_dataloader)
        #print(f'runnnig step: {train_running_step}')

        model.train();
        train_mse   = 0
        for train_idx, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            if model_type == 'DNN':
                labels_state = labels.to(device)

            else:
                labels_state = labels.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
                #print(outputs[0][:, -1, :].unsqueeze(1).shape, labels_state.shape, labels_behavior.shape)
                
                if model_type == 'DNN':
                    loss_mse = criterionMSE(outputs, labels_state)     # 기체 상태 MSE 예측
                else:
                    if model_type[:7] == 'Stacked': 
                        outputs = outputs.unsqueeze(1)
                    loss_mse = criterionMSE(outputs, labels_state)

                loss = loss_mse
            loss.backward()
            optimizer.step()

            train_mse += loss_mse.item()
        train_mse   = train_mse/train_running_step

        # validation Procedure
        val_running_step = len(val_dataloader)
        #print(f'runnnig step: {val_running_step}')

        model.eval();
        val_mse   = 0
        with torch.no_grad():
            for val_idx, (inputs, labels) in enumerate(val_dataloader):
                inputs = inputs.to(device)
                if model_type == 'DNN':
                    labels_state = labels.to(device)
                else:
                    labels_state = labels.to(device)

                outputs = model(inputs)
                if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
                if model_type == 'DNN':
                    loss_mse = criterionMSE(outputs, labels_state)     # 기체 상태 MSE 예측
                else:
                    if model_type[:7] == 'Stacked':  
                        outputs = outputs.unsqueeze(1)
                    loss_mse = criterionMSE(outputs, labels_state)

                val_mse += loss_mse.item()
 
        val_mse   = val_mse/val_running_step

        if val_mse < best_total:
            best_epoch = epoch
            #print(f'\t Best MSE changed [{round(best_total, 5)} ---> {round(val_total, 5)}]')
            save('best_dji', epoch, SAVE_PATH, model, optimizer)     
            best_total = val_mse
        save('last_dji', epoch, SAVE_PATH, model, optimizer)
        #print(f'\t Validaition [Current|Best]: [{round(val_mse, 5)}| {round(best_total, 5)}]')

        res['train_mse'].append(train_mse)
        
        res['val_mse'].append(val_mse)

        res['best_total'].append(best_total)
        pd.DataFrame(res).to_csv(SAVE_PATH+'/dji_res.csv', index=False)

        if schedule:
            scheduler.step()
    print('best_Epoch: ', best_epoch)

def test(model, SAVE_PATH, model_type, test_dataloader, device, column_idx):
    model.load_state_dict(torch.load(os.path.join(SAVE_PATH, 'best_dji.path.tar'))['model_state_dict'])
    
    criterion = nn.MSELoss()
    softmax   = nn.Softmax()
    model.eval();
    #test_results = {}
    mse = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            if model_type == 'DNN':
                labels_state = labels.to(device)
            else:
                labels_state = labels.to(device)
                 
            outputs = model(inputs)
            if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
            outputs=outputs.squeeze(1)
            mse.extend(((outputs - labels_state.squeeze(1))**2).detach().cpu().numpy())

    mse = np.array(mse).mean(axis=0)

#     results = {}
#     for idx in column_idx:
#         results[idx] = [mse[column_idx[idx]].mean()]
    
#     results_df = pd.DataFrame(results)
#     results_df.to_csv(os.path.join(SAVE_PATH, 'test_dji.csv'), index=False)
#     print('test save path: ', os.path.join(SAVE_PATH, 'test_dji.csv'))
    print('\n####################################################################\n')
    return mse
        
    
                          
def experiment(model_type, hid_dim, n_layers, window_size, pred_len, EPOCH, schedule, train_dataloader, val_dataloader, test_dataloader, column_idx):
    IN_FEAT     = 41
    OUT_FEAT    = 36
    IN_SEQ      = window_size
    OUT_SEQ     = pred_len
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if model_type == 'DNN':
        flatten = True
        model = DNN(input_size = IN_FEAT*IN_SEQ, hidden_size = hid_dim, output_size = OUT_FEAT*OUT_SEQ).to(device)
    else: 
        flatten = False
        if model_type[:7] == 'Stacked':
            model = Stacked(model_type[7:], IN_FEAT, hid_dim, OUT_FEAT, 4).to(device)
        elif model_type[:7] == 'Seq2Seq':
            model = Seq2Seq(input_dim=IN_FEAT, output_dim=OUT_FEAT, hid_dim=hid_dim, n_layers=n_layers, model_type=model_type[8:], pred_len=OUT_SEQ).to(device)
        else:
            model = eval('nn.{}(IN_FEAT, OUT_FEAT, n_layers, batch_first=True).to(device)'.format(model_type))
    
    SAVE_PATH   = os.path.join('weights', '_'.join([model_type, str(window_size), str(pred_len), str(hid_dim), str(n_layers)]))

    if schedule:
        SAVE_PATH   = os.path.join(['weights', 'scheculer', '_'.join([model_type, window_size, pred_len])])
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    #train(model, schedule, EPOCH, train_dataloader, val_dataloader, SAVE_PATH, model_type, device)
    
    mse = test(model, SAVE_PATH, model_type, test_dataloader, device, column_idx)
    
    if device == 'cuda':
        torch.cuda.empty_cache()
        
    return mse

In [ ]:
models = ['DNN', 'Seq2Seq_RNN', 'Seq2Seq_LSTM', 'Seq2Seq_GRU']
epoch = 100
hid_dim = 256
n_layers=4
window_size = 10
pred_len = 1
schedule = False 

usecols = [
    "AAClock",
    "IMU_ATTI(0).Longitude",
    "IMU_ATTI(0).Latitude",
    "IMU_ATTI(0).alti.D",
    "IMU_ATTI(0).roll.C",
    "IMU_ATTI(0).pitch.C",
    "IMU_ATTI(0).yaw.C",
    "IMU_ATTI(0).accelX",
    "IMU_ATTI(0).accelY",
    "IMU_ATTI(0).accelZ",
    "IMU_ATTI(0).gyroX",
    "IMU_ATTI(0).gyroY",
    "IMU_ATTI(0).gyroZ",
    "IMU_ATTI(0).quatW.D",
    "IMU_ATTI(0).quatX.D",
    "IMU_ATTI(0).quatY.D",
    "IMU_ATTI(0).quatZ.D",
    "IMU_ATTI(0).ison",
    "IMUCalcs(0).PosN.C",
    "IMUCalcs(0).PosE.C",
    "IMUCalcs(0).PosD.C",
    "IMUCalcs(0).velN.C",
    "IMUCalcs(0).velE.C",
    "IMUCalcs(0).velD.C",
    "IMUCalcs(0).ison",
    "GPS.Time",
    "GPS.Long",
    "GPS.Lat",
    "GPS.heightMSL",
    "GPS.hDOP",
    "GPS.pDOP",
    "GPS.numSV",
    "GPS.velN",
    "GPS.velE",
    "GPS.velD",
    "GPS.ison",
    "BatteryInfo.BatCap.D",
    "BatteryInfo.BatTemp.D",
    "BatteryInfo.BatVol.D",
    "BatteryInfo.ison",
    "AirSpeed.windSpeed",
    "AirSpeed.ison",
    "scenario"
]

#X_train, y_train, X_val, y_val, X_test, y_test, column_idx = get_data()
for model_type in models:
    if model_type == 'DNN':
        flatten = True
    else:
        flatten = False
    train_dataloader, val_dataloader, test_dataloader, column_idx = dji_dataloader('dji_normal_processed.csv' , usecols, flatten=flatten)

    experiment(model_type, hid_dim, n_layers, window_size, pred_len, epoch, schedule, train_dataloader, val_dataloader, test_dataloader, column_idx)
    

ison_list: ['AirSpeed.ison', 'BatteryInfo.ison', 'GPS.ison', 'IMUCalcs(0).ison', 'IMU_ATTI(0).ison']
ison_dict: {'AirSpeed.ison': 0, 'BatteryInfo.ison': 5, 'GPS.ison': 11, 'IMUCalcs(0).ison': 20, 'IMU_ATTI(0).ison': 33}
Training
Scenario: [11  8  1 24  6 22 17 13 26 29 31 10  2 14 27 25  4 18 21  9]
Shape of the traninig sample for input: (61122, 10, 41)
Shape of the traninig sample for label: (61122, 1, 36)

Validation
Scenario: [ 7 32  5]
Shape of the traninig sample for input: (8450, 10, 41)
Shape of the traninig sample for label: (8450, 1, 36)

Test
Scenario: [ 3 20 23 12 19 30 15 16 33 28]
Shape of the traninig sample for input: (32173, 10, 41)
Shape of the traninig sample for label: (32173, 1, 36)
train path:  weights\DNN_10_1_256_4/dji_res.csv


 20%|████████▏                                | 20/100 [01:12<04:49,  3.62s/it]

# 센서별 mse 출력

In [6]:
models = ['DNN', 'Seq2Seq_RNN', 'Seq2Seq_LSTM', 'Seq2Seq_GRU']
epoch = 100
hid_dim = 256
n_layers=4
window_size = 10
pred_len = 1
schedule = False 

usecols = [
    "AAClock",
    "IMU_ATTI(0).Longitude",
    "IMU_ATTI(0).Latitude",
    "IMU_ATTI(0).alti.D",
    "IMU_ATTI(0).roll.C",
    "IMU_ATTI(0).pitch.C",
    "IMU_ATTI(0).yaw.C",
    "IMU_ATTI(0).accelX",
    "IMU_ATTI(0).accelY",
    "IMU_ATTI(0).accelZ",
    "IMU_ATTI(0).gyroX",
    "IMU_ATTI(0).gyroY",
    "IMU_ATTI(0).gyroZ",
    "IMU_ATTI(0).quatW.D",
    "IMU_ATTI(0).quatX.D",
    "IMU_ATTI(0).quatY.D",
    "IMU_ATTI(0).quatZ.D",
    "IMU_ATTI(0).ison",
    "IMUCalcs(0).PosN.C",
    "IMUCalcs(0).PosE.C",
    "IMUCalcs(0).PosD.C",
    "IMUCalcs(0).velN.C",
    "IMUCalcs(0).velE.C",
    "IMUCalcs(0).velD.C",
    "IMUCalcs(0).ison",
    "GPS.Time",
    "GPS.Long",
    "GPS.Lat",
    "GPS.heightMSL",
    "GPS.hDOP",
    "GPS.pDOP",
    "GPS.numSV",
    "GPS.velN",
    "GPS.velE",
    "GPS.velD",
    "GPS.ison",
    "BatteryInfo.BatCap.D",
    "BatteryInfo.BatTemp.D",
    "BatteryInfo.BatVol.D",
    "BatteryInfo.ison",
    "AirSpeed.windSpeed",
    "AirSpeed.ison",
    "scenario"
]

#X_train, y_train, X_val, y_val, X_test, y_test, column_idx = get_data()
for model_type in models:
    if model_type == 'DNN':
        flatten = True
    else:
        flatten = False
    train_dataloader, val_dataloader, test_dataloader, column_idx, test_data = dji_dataloader('dji_normal_processed.csv' , usecols, flatten=flatten)

    mse = experiment(model_type, hid_dim, n_layers, window_size, pred_len, epoch, schedule, train_dataloader, val_dataloader, test_dataloader, column_idx)

ison_list: ['AirSpeed.ison', 'BatteryInfo.ison', 'GPS.ison', 'IMUCalcs(0).ison', 'IMU_ATTI(0).ison']
ison_dict: {'AirSpeed.ison': 0, 'BatteryInfo.ison': 5, 'GPS.ison': 11, 'IMUCalcs(0).ison': 20, 'IMU_ATTI(0).ison': 33}

####################################################################



In [33]:
len(mse)

36

In [50]:
column_idx

{'AirSpeed': [[0], ['windSpeed']],
 'BatteryInfo': [[1, 2, 3], ['D', 'D', 'D']],
 'GPS': [[4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  ['Lat',
   'Long',
   'Time',
   'hDOP',
   'heightMSL',
   'numSV',
   'pDOP',
   'velD',
   'velE',
   'velN']],
 'IMUCalcs(0)': [[14, 15, 16, 17, 18, 19], ['C', 'C', 'C', 'C', 'C', 'C']],
 'IMU_ATTI(0)': [[20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35],
  ['Latitude',
   'Longitude',
   'accelX',
   'accelY',
   'accelZ',
   'D',
   'gyroX',
   'gyroY',
   'gyroZ',
   'C',
   'D',
   'D',
   'D',
   'D',
   'C',
   'C']]}

In [43]:
sensors = test_data.keys().drop(['AirSpeed.ison', 'BatteryInfo.ison', 'GPS.ison', 'IMUCalcs(0).ison', 'IMU_ATTI(0).ison', 'scenario'])

In [44]:
a = pd.DataFrame(mse).T
a.to_csv('weights/dji_all_sensors.csv', index=False, header=sensors)

In [ ]:
new_mse = mse[:20]

In [51]:
print(sensors[:20][mse[:20].argmin()], sensors[:20][mse[:20].argmax()])

GPS.Long BatteryInfo.BatCap.D


In [49]:
sensors

Index(['AirSpeed.windSpeed', 'BatteryInfo.BatCap.D', 'BatteryInfo.BatTemp.D',
       'BatteryInfo.BatVol.D', 'GPS.Lat', 'GPS.Long', 'GPS.Time', 'GPS.hDOP',
       'GPS.heightMSL', 'GPS.numSV', 'GPS.pDOP', 'GPS.velD', 'GPS.velE',
       'GPS.velN', 'IMUCalcs(0).PosD.C', 'IMUCalcs(0).PosE.C',
       'IMUCalcs(0).PosN.C', 'IMUCalcs(0).velD.C', 'IMUCalcs(0).velE.C',
       'IMUCalcs(0).velN.C', 'IMU_ATTI(0).Latitude', 'IMU_ATTI(0).Longitude',
       'IMU_ATTI(0).accelX', 'IMU_ATTI(0).accelY', 'IMU_ATTI(0).accelZ',
       'IMU_ATTI(0).alti.D', 'IMU_ATTI(0).gyroX', 'IMU_ATTI(0).gyroY',
       'IMU_ATTI(0).gyroZ', 'IMU_ATTI(0).pitch.C', 'IMU_ATTI(0).quatW.D',
       'IMU_ATTI(0).quatX.D', 'IMU_ATTI(0).quatY.D', 'IMU_ATTI(0).quatZ.D',
       'IMU_ATTI(0).roll.C', 'IMU_ATTI(0).yaw.C'],
      dtype='object')

# train_data로 뭘 하는 코드

In [11]:
usecols = [
    "AAClock",
    "IMU_ATTI(0).Longitude",
    "IMU_ATTI(0).Latitude",
    "IMU_ATTI(0).alti.D",
    "IMU_ATTI(0).roll.C",
    "IMU_ATTI(0).pitch.C",
    "IMU_ATTI(0).yaw.C",
    "IMU_ATTI(0).accelX",
    "IMU_ATTI(0).accelY",
    "IMU_ATTI(0).accelZ",
    "IMU_ATTI(0).gyroX",
    "IMU_ATTI(0).gyroY",
    "IMU_ATTI(0).gyroZ",
    "IMU_ATTI(0).quatW.D",
    "IMU_ATTI(0).quatX.D",
    "IMU_ATTI(0).quatY.D",
    "IMU_ATTI(0).quatZ.D",
    "IMU_ATTI(0).ison",
    "IMUCalcs(0).PosN.C",
    "IMUCalcs(0).PosE.C",
    "IMUCalcs(0).PosD.C",
    "IMUCalcs(0).velN.C",
    "IMUCalcs(0).velE.C",
    "IMUCalcs(0).velD.C",
    "IMUCalcs(0).ison",
    "GPS.Time",
    "GPS.Long",
    "GPS.Lat",
    "GPS.heightMSL",
    "GPS.hDOP",
    "GPS.pDOP",
    "GPS.numSV",
    "GPS.velN",
    "GPS.velE",
    "GPS.velD",
    "GPS.ison",
    "BatteryInfo.BatCap.D",
    "BatteryInfo.BatTemp.D",
    "BatteryInfo.BatVol.D",
    "BatteryInfo.ison",
    "AirSpeed.windSpeed",
    "AirSpeed.ison",
    "scenario"
]

flatten = False
train_dataloader, val_dataloader, test_dataloader, column_idx, train_data = dji_dataloader('dji_normal_processed.csv' , usecols, flatten=flatten)
train_data.keys()

ison_list: ['AirSpeed.ison', 'BatteryInfo.ison', 'GPS.ison', 'IMUCalcs(0).ison', 'IMU_ATTI(0).ison']
ison_dict: {'AirSpeed.ison': 0, 'BatteryInfo.ison': 5, 'GPS.ison': 11, 'IMUCalcs(0).ison': 20, 'IMU_ATTI(0).ison': 33}
Training
Scenario: [ 6 14 20 32 21  2  4 16 26 27 31 28 10  5 22 33 18  3 23 17]
Shape of the traninig sample for input: (63706, 10, 41)
Shape of the traninig sample for label: (63706, 1, 36)

Validation
Scenario: [19 11  8]
Shape of the traninig sample for input: (9410, 10, 41)
Shape of the traninig sample for label: (9410, 1, 36)

Test
Scenario: [12 30 24  9  7 25  1 13 15 29]
Shape of the traninig sample for input: (28629, 10, 41)
Shape of the traninig sample for label: (28629, 1, 36)


Index(['AirSpeed.ison', 'AirSpeed.windSpeed', 'BatteryInfo.BatCap.D',
       'BatteryInfo.BatTemp.D', 'BatteryInfo.BatVol.D', 'BatteryInfo.ison',
       'GPS.Lat', 'GPS.Long', 'GPS.Time', 'GPS.hDOP', 'GPS.heightMSL',
       'GPS.ison', 'GPS.numSV', 'GPS.pDOP', 'GPS.velD', 'GPS.velE', 'GPS.velN',
       'IMUCalcs(0).PosD.C', 'IMUCalcs(0).PosE.C', 'IMUCalcs(0).PosN.C',
       'IMUCalcs(0).ison', 'IMUCalcs(0).velD.C', 'IMUCalcs(0).velE.C',
       'IMUCalcs(0).velN.C', 'IMU_ATTI(0).Latitude', 'IMU_ATTI(0).Longitude',
       'IMU_ATTI(0).accelX', 'IMU_ATTI(0).accelY', 'IMU_ATTI(0).accelZ',
       'IMU_ATTI(0).alti.D', 'IMU_ATTI(0).gyroX', 'IMU_ATTI(0).gyroY',
       'IMU_ATTI(0).gyroZ', 'IMU_ATTI(0).ison', 'IMU_ATTI(0).pitch.C',
       'IMU_ATTI(0).quatW.D', 'IMU_ATTI(0).quatX.D', 'IMU_ATTI(0).quatY.D',
       'IMU_ATTI(0).quatZ.D', 'IMU_ATTI(0).roll.C', 'IMU_ATTI(0).yaw.C',
       'scenario'],
      dtype='object')

In [10]:
df = train_data['IMU_ATTI(0).ison']
df.unique()

array([0], dtype=int64)